In [23]:
import pandas as pd
import numpy as np
import torch

# make jupytre relode modules when they change
%load_ext autoreload
%autoreload 2

In [99]:
# csv file path
path = ''

In [100]:
# read csv file to pandas dataframe
df = pd.read_csv(path)

In [4]:
df.iloc[0:10].to_numpy().shape

(10, 13)

In [98]:
# Create dataset class
class Dataset:
    def __init__(self, csv_path, num_time_steps_before, num_time_steps_after):
        self.df = pd.read_csv(csv_path)
        self.num_time_steps_before = num_time_steps_before
        self.num_time_steps_after = num_time_steps_after

    def __len__(self):
        return len(self.df) - self.num_time_steps_before - self.num_time_steps_after

    def __getitem__(self, idx):

        # Get a slice of the dataframe
        df = self.df.iloc[idx:idx+self.num_time_steps_before+self.num_time_steps_after].copy()

        # drop timestamp column
        df = df.drop(columns=['timestamp'])

        # Extract data
        past_inputs = df.iloc[:self.num_time_steps_before][[
            'flex_myobrick_pwm', 
            'extend_myobrick_pwm'
        ]].to_numpy().astype(np.float32)
        past_outputs = df.iloc[:self.num_time_steps_before][
            'angle'
        ].to_numpy().astype(np.float32)
        past_interals = df.iloc[:self.num_time_steps_before][[
            'safety_switch_pressed',
            'flex_myobrick_pos_encoder',
            'flex_myobrick_torque_encoder',
            'flex_myobrick_current',
            'flex_myobrick_in_running_state',
            'extend_myobrick_pos_encoder',
            'extend_myobrick_torque_encoder',
            'extend_myobrick_current',
            'extend_myobrick_in_running_state'
        ]].to_numpy().astype(np.float32)
        future_desired_outputs = df.iloc[self.num_time_steps_before:self.num_time_steps_before+self.num_time_steps_after][
            'angle'
        ].to_numpy().astype(np.float32)
        future_predicted_inputs = df.iloc[self.num_time_steps_before:self.num_time_steps_before+self.num_time_steps_after][[
            'flex_myobrick_pwm',
            'extend_myobrick_pwm'
        ]].to_numpy().astype(np.float32)


        # convert to tensors
        past_inputs = torch.from_numpy(past_inputs).permute(1, 0)
        past_outputs = torch.from_numpy(past_outputs).unsqueeze(0)
        past_interals = torch.from_numpy(past_interals).permute(1, 0)
        future_desired_outputs = torch.from_numpy(future_desired_outputs).unsqueeze(0)
        future_predicted_inputs = torch.from_numpy(future_predicted_inputs).permute(1, 0)

        return past_inputs, past_outputs, past_interals, future_desired_outputs, future_predicted_inputs

    

In [91]:
# Create dataset
dataset = Dataset(path, 20, 20)

# Create dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

In [94]:
# Get a batch
past_inputs, past_outputs, past_interals, future_desired_outputs, future_predicted_inputs = next(iter(dataloader))

In [63]:
# Import pytorch_lightning model
import pytorch_lightning as pl


import sys
sys.path.append('../')

from system_controller_model import SystemControllerModel

In [72]:
model = SystemControllerModel(
    num_past_time_steps=20,
    num_future_time_steps=20,
    num_input_vars=2,
    num_output_vars=1,
    num_internal_vars=9
)

In [77]:
# train model and print loss
trainer = pl.Trainer(max_epochs=3)
trainer.fit(model, dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/nils/miniconda3/envs/i2dl/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name                        | Type   | Params
-------------------------------------------------------
0 | past_conv1                  | Conv1d | 185   
1 | past_conv2                  | Conv1d | 160   
2 | future_desired_output_conv1 | Conv1d | 15    
3 | future_desired_output_conv2 | Conv1d | 55    
4 | fc1                         | Linear | 72.4 K
5 | fc2                         | Linear | 16.0 K
-------------------------------------------------------
88.9 K    Trainable params
0         Non-trainable params
88.9 K    Total params
0.355     Total estimated model params size (MB)
/home/n

Epoch 2: 100%|██████████| 230/230 [00:03<00:00, 71.19it/s, loss=0, v_num=2]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 230/230 [00:03<00:00, 71.00it/s, loss=0, v_num=2]


In [75]:
trainer.logged_metrics['train_loss']

tensor(0.)

In [90]:
# Test model
model.eval()
with torch.no_grad():
    past_inputs, past_outputs, past_interals, future_desired_outputs, future_predicted_inputs = next(iter(dataloader))
    future_predicted_outputs_hat = model(past_inputs, past_outputs, past_interals, future_desired_outputs)
    

# Plot results
import matplotlib.pyplot as plt



In [88]:
future_predicted_outputs.min()

tensor(0.)